In [1]:
import re 
import sys 
import numpy as np 
sys.path.append('/home/work/yuna/VLMEval') 
from dataset.dataloader import PostProcessor # contractions, numbers, punctuations
from analysis.pilot.analysis import vqa_score, str_to_list 
from analysis.utils.question_type_mapper import question_type 
from typing import List, Any
import pandas as pd 
from glob import glob 
import csv
import json
import os

### READ SELECTED QUESIOTNS  
qs = pd.read_csv('/home/work/yuna/HPA/eda/csv/ACL_mmstar_questions - s1.csv', header=None) # selected questions for mmstar 
### READ HUMAN RESULTS 
df = pd.read_excel("/home/work/yuna/HPA/results/human-raw/VQA-SPUB Pilot Experiment Pt.4 (Responses).xlsx")
df = df.melt(id_vars=['Timestamp', "Name or Initials 이름 또는 이니셜 ", "Score"]).rename(columns={'variable': 'question', 'value': 'answer', "이름 또는 이니셜 Name or Initials ": "subj"})
df['hqid'] = df['question'].factorize()[0] 
selected = pd.read_csv('/home/work/yuna/HPA/eda/csv/mmstar_questions_blind_descending.csv').rename(columns={'Unnamed: 0': 'qid'})
selected.index.name = 'hqid'
selected = selected.reset_index()
print(len(df))
df = pd.merge(selected, df, on =['hqid'], how='right', suffixes=('_gt', ''))
df['correct'] = df['answer_gt'] == df['answer']
len(df.qid.unique())

In [99]:
results = pd.merge(df.groupby(["category",	"l2_category"]).agg({'correct': 'mean', 'question':'nunique'}).reset_index(),
        blind.groupby(["category",	"l2_category"])['correct'].mean().reset_index(), 
        on=["category",	"l2_category"], suffixes=('_human', '_model')
        )
        
results['diff'] = np.abs(results['correct_model'] - results['correct_human'])
results.sort_values(by=['diff']) 

,category,l2_category,correct_human,question,correct_model,diff
8,instance reasoning,single-instance reasoning,0.343750,4,0.340909,0.002841
13,math,numeric commonsense and calculation,0.250000,1,0.242857,0.007143
10,logical reasoning,common reasoning,0.240385,13,0.232877,0.007508
3,fine-grained perception,localization,0.125000,1,0.083333,0.041667
2,coarse perception,image style & quality,0.250000,2,0.291667,0.041667
0,coarse perception,image emotion,0.375000,4,0.289474,0.085526
15,science & technology,biology & chemistry & physics,0.187500,2,0.100000,0.087500
5,fine-grained perception,recognition,0.321429,7,0.208333,0.113095
11,logical reasoning,diagram reasoning,0.166667,6,0.280488,0.113821
12,math,geometry,0.270000,25,0.390000,0.120000


In [84]:
blind = mmstar[mmstar['blind'] == True]
blind = blind.rename(columns={"index": 'qid'})
blind['qid'] = blind['qid'].astype(int)
blind = blind[blind['qid'].isin(df.qid.unique())]
len(blind.qid.unique())

# Model resutls 

In [ ]:
results_by_categories = pd.merge(
    mmstar[mmstar['blind'] == False].groupby(['category', 'l2_category']).correct.mean().reset_index().sort_values(by=['correct'], ascending=False), 
    mmstar[mmstar['blind'] == True].groupby(['category', 'l2_category']).correct.mean().reset_index().sort_values(by=['correct'], ascending=False), 
    on=['category', 'l2_category'], 
    how='inner', 
    suffixes=('', '_blind') 
)

results_by_categories['diff'] = results_by_categories['correct'] - results_by_categories['correct_blind'] 
results_by_categories.sort_values(by=['diff'], ascending=False)

,category,l2_category,correct,correct_blind,diff
0,coarse perception,image style & quality,0.610357,0.301907,0.308450
3,instance reasoning,single-instance reasoning,0.560606,0.259871,0.300735
2,coarse perception,image emotion,0.562105,0.308901,0.253205
7,fine-grained perception,localization,0.438333,0.189583,0.248750
1,instance reasoning,cross-instance relation reasoning,0.563364,0.318182,0.245182
6,coarse perception,image scene and topic,0.443238,0.220917,0.222321
5,logical reasoning,diagram reasoning,0.453247,0.235450,0.217797
4,instance reasoning,cross-instance attribute reasoning,0.462279,0.269663,0.192616
8,logical reasoning,common reasoning,0.433522,0.250476,0.183046
10,math,statistical reasoning,0.399502,0.234884,0.164618


In [19]:
qs = mmstar[mmstar['blind'] == True].groupby(['category', 'l2_category', 'question', 'answer']).correct.mean().reset_index().sort_values(by=['correct'], ascending=False)
qs.index.name = 'qid'
qs.to_csv('./csv/mmstar_questions_blind_desceanding.csv')
qs = qs.reset_index() 

In [15]:
mmstar.groupby(['category', 'l2_category']).question.nunique()

category                 l2_category                            
coarse perception        image emotion                               19
                         image scene and topic                      135
                         image style & quality                       21
fine-grained perception  localization                                39
                         object counting                             92
                         recognition                                118
instance reasoning       cross-instance attribute reasoning          75
                         cross-instance relation reasoning           48
                         single-instance reasoning                   93
logical reasoning        code & sequence reasoning                   38
                         common reasoning                            97
                         diagram reasoning                          108
math                     geometry                                   116

In [5]:
mmstar = [] 

for file in glob('../results/mmstar/*.jsonl'): 
    
    with open(file, 'r') as f:
        data = [json.loads(line) for line in f]
    df = pd.DataFrame(data)
    df['model'] = '_'.join(file.replace('_224', '').split('/')[-1].split('.jsonl')[0].split('_')[1:])
    mmstar.append(df) 

mmstar = pd.concat(mmstar, axis=0) 
mmstar['blind'] = mmstar['model'].str.contains('-blind') 
mmstar['model'] = mmstar['model'].str.replace('-blind', '') 
mmstar['correct'] = mmstar['output'].str.strip().str.lower() == mmstar['answer'].str.strip().str.lower()
results = mmstar.groupby(['model', 'category', 'blind'])['correct'].mean().reset_index().pivot(index=['model', 'blind'], columns='category', values='correct') 
print(results.dropna(axis=0).reset_index().model.unique())
results 

['InternVL2_5-1B' 'InternVL2_5-2B' 'InternVL2_5-4B' 'InternVL2_5-8B'
 'InternVL3-1B' 'InternVL3-2B' 'Qwen2.5-VL-3B-Instruct'
 'Qwen2.5-VL-7B-Instruct' 'Qwen3-VL-4B-Instruct' 'Qwen3-VL-8B-Instruct'
 'gemma-3-12b-it' 'gemma-3-4b-it']


category                             coarse perception  \
model                         blind                      
InternVL2_5-1B                False              0.538   
                              True               0.256   
InternVL2_5-2B                False              0.676   
                              True               0.448   
InternVL2_5-4B                False              0.708   
                              True               0.284   
InternVL2_5-8B                False              0.352   
                              True               0.000   
InternVL3-1B                  False              0.448   
                              True               0.448   
InternVL3-2B                  False              0.488   
                              True               0.272   
Qwen2-VL-2B-Instruct-bnb-4bit False              0.000   
                              True               0.000   
Qwen2-VL-7B-Instruct-bnb-4bit False              0.000   
                              True               0.000   
Qwen2.5-VL-3B-Instruct        False              0.456   
                              True               0.324   
Qwen2.5-VL-7B-Instruct        False              0.500   
                              True               0.260   
Qwen3-VL-4B-Instruct          False              0.712   
Qwen3-VL-8B-Instruct          False              0.752   
                              True               0.256   
gemma-3-12b-it                False              0.704   
                              True               0.340   
gemma-3-4b-it                 False              0.640   
                              True               0.304   

category                             fine-grained perception  \
model                         blind                            
InternVL2_5-1B                False                    0.326   
                              True                     0.200   
InternVL2_5-2B                False                    0.404   
                              True                     0.320   
InternVL2_5-4B                False                    0.512   
                              True                     0.180   
InternVL2_5-8B                False                    0.350   
                              True                     0.000   
InternVL3-1B                  False                    0.388   
                              True                     0.388   
InternVL3-2B                  False                    0.424   
                              True                     0.240   
Qwen2-VL-2B-Instruct-bnb-4bit False                    0.000   
                              True                     0.000   
Qwen2-VL-7B-Instruct-bnb-4bit False                      NaN   
                              True                       NaN   
Qwen2.5-VL-3B-Instruct        False                    0.364   
                              True                     0.196   
Qwen2.5-VL-7B-Instruct        False                    0.420   
                              True                     0.240   
Qwen3-VL-4B-Instruct          False                    0.480   
Qwen3-VL-8B-Instruct          False                    0.560   
                              True                     0.192   
gemma-3-12b-it                False                    0.416   
                              True                     0.232   
gemma-3-4b-it                 False                    0.340   
                              True                     0.244   

category                             instance reasoning  logical reasoning  \
model                         blind                                          
InternVL2_5-1B                False               0.458           0.268000   
                              True                0.288           0.224000   
InternVL2_5-2B                False               0.608           0.424000   
                              True                0.420           0.28